In [34]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import sys
sys.path.append('../.') 
from src.graph.create_osmnx_graph import OSMnxGraph
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import networkx as nx
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
df_accidents = pd.read_csv("../data/wypadki-pl/accidents.csv")
df_accidents.drop(df_accidents[(df_accidents['mie_nazwa'] != 'Warszawa')].index, inplace=True)
df_accidents.drop(columns='uczestnicy', inplace=True)
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(df_accidents['wsp_gps_x'], df_accidents['wsp_gps_y'])]
gdf_acc = gpd.GeoDataFrame(df_accidents, geometry=geometry)
gdf_acc.drop(columns=['wsp_gps_x', 'wsp_gps_y'], inplace=True)
G =ox.graph.graph_from_place("Warsaw, Poland", network_type="drive")
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
gdf_nodes

,y,x,street_count,highway,ref,geometry
osmid,,,,,,
26063726,52.152787,21.017454,5,NaN,NaN,POINT (21.01745 52.15279)
26063848,52.157570,20.991392,4,NaN,NaN,POINT (20.99139 52.15757)
26063857,52.166411,20.992301,3,NaN,NaN,POINT (20.99230 52.16641)
26063923,52.170803,20.992247,3,NaN,NaN,POINT (20.99225 52.17080)
26083887,52.176026,20.995141,3,NaN,NaN,POINT (20.99514 52.17603)
...,...,...,...,...,...,...
11795306888,52.284337,21.051939,1,NaN,NaN,POINT (21.05194 52.28434)
11795306892,52.284337,21.052441,3,NaN,NaN,POINT (21.05244 52.28434)
11795306899,52.285418,21.051583,1,NaN,NaN,POINT (21.05158 52.28542)


## Edges EDA

Wszystkie zgromadzone cechy potraktowano analogicznie jak w pracy magisterskiej z road networks za wyjątkiem atrybutów: psv, service, busway, bicycle, cycleway i surface, które nie pojawiły się w zestawieniu.

Preprocessing obejmował:
* lanes - dodanie domyślnej wartości 2
* maxspeed - dodanie domyślnej wartości 50
* width - dodanie domyślnej wartości 2.0
* pozostałe - dodanie wartości "unspecified"

Podjęto też próbę uzupełnienia prędkości z wykorzystaniem metody oferowanej przez OSMNx.



In [5]:
gdf_edges

osmid  \
u           v           key                                                    
26063726    31717945    0                                            4895484   
            29273144    0    [1014952979, 1014952978, 1007148203, 148749604]   
26063848    1391593855  0       [13982288, 125098266, 896320940, 1203516007]   
            258495263   0                                           23853023   
            26083906    0                                          715903941   
...                                                                      ...   
11795306899 11795306883 0                                         1270353666   
11795306921 11795306923 0                                         1270353669   
11795306923 11795306892 0                                         1270353667   
            11795306888 0                                         1270353667   
            11795306921 0                                         1270353669   

                             oneway   lanes                        name  \
u           v           key                                               
26063726    31717945    0      True       4                    Poleczki   
            29273144    0      True  [4, 3]                    Puławska   
26063848    1391593855  0     False       2  [Nowy Służewiec, Wyczółki]   
            258495263   0     False       2                     Zatorze   
            26083906    0     False     NaN                    Wyczółki   
...                             ...     ...                         ...   
11795306899 11795306883 0     False     NaN                         NaN   
11795306921 11795306923 0     False     NaN                         NaN   
11795306923 11795306892 0     False     NaN                         NaN   
            11795306888 0     False     NaN                         NaN   
            11795306921 0     False     NaN                         NaN   

                                   highway       reversed   length maxspeed  \
u           v           key                                                   
26063726    31717945    0        secondary          False   25.021      NaN   
            29273144    0          primary          False  336.457       60   
26063848    1391593855  0      residential  [False, True]  996.078      NaN   
            258495263   0         tertiary          False  353.996      NaN   
            26083906    0      residential          False   66.590      NaN   
...                                    ...            ...      ...      ...   
11795306899 11795306883 0    living_street           True   89.591       20   
11795306921 11795306923 0    living_street          False   14.146       20   
11795306923 11795306892 0    living_street          False   23.305       20   
            11795306888 0    living_street           True   14.195       20   
            11795306921 0    living_street           True   14.146       20   

                                                                      geometry  \
u           v           key                                                      
26063726    31717945    0    LINESTRING (21.01745 52.15279, 21.01780 52.15271)   
            29273144    0    LINESTRING (21.01745 52.15279, 21.01745 52.152...   
26063848    1391593855  0    LINESTRING (20.99139 52.15757, 20.99129 52.157...   
            258495263   0    LINESTRING (20.99139 52.15757, 20.99127 52.157...   
            26083906    0    LINESTRING (20.99139 52.15757, 20.99147 52.157...   
...                                                                        ...   
11795306899 11795306883 0    LINESTRING (21.05158 52.28542, 21.05159 52.285...   
11795306921 11795306923 0    LINESTRING (21.05205 52.28449, 21.05205 52.284...   
11795306923 11795306892 0    LINESTRING (21.05212 52.28437, 21.05212 52.284...   
            11795306888 0    LINESTRING (21.05212 52.28437, 21.05211 52.284...   
            11795306921 0    LINESTRING (2

In [6]:
gdf_edges.replace('NaN', np.nan, inplace=True)
gdf_edges.isna().sum()

osmid           0
oneway          0
lanes       26120
name         6836
highway         0
reversed        0
length          0
maxspeed    24852
geometry        0
access      43632
junction    43062
bridge      43593
tunnel      44060
ref         43188
width       43607
dtype: int64

In [7]:
gdf_edges[~gdf_edges['ref'].isna()]

osmid  \
u           v          key                                                
26385831    339919381  0              [481567944, 481567945, 481566053]   
26405568    4068485239 0                                      481568066   
26405569    26405570   0                                      481044837   
26405570    270207773  0                                      481044837   
26515507    1273140010 0    [4811536, 356292123, 1178290413, 111823182]   
...                                                                 ...   
11378326108 3251228059 0                                      112402152   
11378326109 2281845699 0                          [353063348, 92537309]   
            1277638280 0                                      247486791   
11542773424 1837412373 0                                     1083286709   
11542773425 8068601798 0                                     1245456480   

                            oneway   lanes                     name  \
u           v          key                                            
26385831    339919381  0      True  [4, 3]                 Puławska   
26405568    4068485239 0      True       5                 Puławska   
26405569    26405570   0      True       3                 Puławska   
26405570    270207773  0      True       3                 Puławska   
26515507    1273140010 0      True       3                  Sasanki   
...                            ...     ...                      ...   
11378326108 3251228059 0      True       2      Aleje Jerozolimskie   
11378326109 2281845699 0      True       3  Aleja Obrońców Warszawy   
            1277638280 0      True       3      Aleje Jerozolimskie   
11542773424 1837412373 0      True       3                Modlińska   
11542773425 8068601798 0      True       3                Modlińska   

                                 highway reversed   length   maxspeed  \
u           v          key                                              
26385831    339919381  0         primary    False  296.250         60   
26405568    4068485239 0         primary    False   12.990         60   
26405569    26405570   0         primary    False  138.731         60   
26405570    270207773  0         primary    False  257.293         60   
26515507    1273140010 0    primary_link    False  319.656         50   
...                                  ...      ...      ...        ...   
11378326108 3251228059 0       secondary    False  250.890        NaN   
11378326109 2281845699 0         primary    False  680.450  [70, 100]   
            1277638280 0    primary_link    False  107.220         50   
11542773424 1837412373 0         primary    False  108.124         60   
11542773425 8068601798 0         primary    False   19.984         60   

                                                                     geometry  \
u           v          key                                                      
26385831    339919381  0    LINESTRING (21.01785 52.13235, 21.01787 52.132...   
26405568    4068485239 0    LINESTRING (21.01812 52.13231, 21.01811 52.13242)   
26405569    26405570   0    LINESTRING (21.01798 52.12779, 21.01799 52.127...   
26405570    270207773  0    LINESTRING (21.01801 52.12654, 21.01803 52.125...   
26515507    1273140010 0    LINESTRING (20.97898 52.18332, 20.97918 52.183...   
...                                                                       ...   
11378326108 3251228059 0    LINESTRING (20.90997 52.18723, 20.91218 52.187...   
11378326109 2281845699 0    LINESTRING (20.91982 52.19051, 20.91888 52.189...   
            1277638280 0    LINESTRING (20.91982 52.19051, 20.91927 52.190...   
11542773424 1837412373 0    LINESTRING (20.95310 52.34199, 20.95333 52.341...   
11542773425 8068601798 0    LINESTRING (20.95271 52.34232, 20.95288 52.34218)   

                           access junction bridge tunnel  ref width  
u           v          key                                           
26385831    339919381 

In [113]:
G_prim = ox.add_edge_speeds(G)
nodes_prim, edges_prim = ox.graph_to_gdfs(G_prim)

In [114]:
edges_prim

osmid  \
u           v           key                                                    
26063726    31717945    0                                            4895484   
            29273144    0    [1014952979, 1014952978, 1007148203, 148749604]   
26063848    1391593855  0       [13982288, 125098266, 896320940, 1203516007]   
            258495263   0                                           23853023   
            26083906    0                                          715903941   
...                                                                      ...   
11795306899 11795306883 0                                         1270353666   
11795306921 11795306923 0                                         1270353669   
11795306923 11795306892 0                                         1270353667   
            11795306888 0                                         1270353667   
            11795306921 0                                         1270353669   

                             oneway   lanes                        name  \
u           v           key                                               
26063726    31717945    0      True       4                    Poleczki   
            29273144    0      True  [3, 4]                    Puławska   
26063848    1391593855  0     False       2  [Wyczółki, Nowy Służewiec]   
            258495263   0     False       2                     Zatorze   
            26083906    0     False     NaN                    Wyczółki   
...                             ...     ...                         ...   
11795306899 11795306883 0     False     NaN                         NaN   
11795306921 11795306923 0     False     NaN                         NaN   
11795306923 11795306892 0     False     NaN                         NaN   
            11795306888 0     False     NaN                         NaN   
            11795306921 0     False     NaN                         NaN   

                                   highway       reversed   length  speed_kph  \
u           v           key                                                     
26063726    31717945    0        secondary          False   25.021       50.8   
            29273144    0          primary          False  336.457       60.0   
26063848    1391593855  0      residential  [False, True]  996.078       40.5   
            258495263   0         tertiary          False  353.996       48.2   
            26083906    0      residential          False   66.590       40.5   
...                                    ...            ...      ...        ...   
11795306899 11795306883 0    living_street           True   89.591       20.0   
11795306921 11795306923 0    living_street          False   14.146       20.0   
11795306923 11795306892 0    living_street          False   23.305       20.0   
            11795306888 0    living_street           True   14.195       20.0   
            11795306921 0    living_street           True   14.146       20.0   

                            maxspeed  \
u           v           key            
26063726    31717945    0        NaN   
            29273144    0         60   
26063848    1391593855  0        NaN   
            258495263   0        NaN   
            26083906    0        NaN   
...                              ...   
11795306899 11795306883 0         20   
11795306921 11795306923 0         20   
11795306923 11795306892 0         20   
            11795306888 0         20   
            11795306921 0         20   

                                                                      geometry  \
u           v           key                                                      
26063726    31717945    0    LINESTRING (21.01745 52.15279, 21.01780 52.15271)   
            29273144    0    LINESTRING (21.01745 52.15279, 21.01745 52.152...   
26063848    1391593855  0    LINESTRING (20.99139 52.15757, 20.99129 52.157...   
            258495263   0    LINESTRING (20.99139 52.15757, 20.99127 

In [115]:
edges_prim.replace('NaN', np.nan, inplace=True)
edges_prim.isna().sum()

osmid            0
oneway           0
lanes        26120
name          6836
highway          0
reversed         0
length           0
speed_kph        0
maxspeed     24852
geometry         0
access       43632
junction     43062
bridge       43593
tunnel       44060
ref          43188
width        43607
dtype: int64

Dodanie maxspeed za pomocą OSMNx nie zrobiło zupełnie nic, więc wypełniono brakujące wartości prędkością 50.

In [8]:
gdf_edges['maxspeed'] = gdf_edges['maxspeed'].fillna(50)
gdf_edges['width'] = pd.to_numeric(gdf_edges['width'], errors='coerce').fillna(2.0).astype(float)
gdf_edges['lanes'] = gdf_edges['lanes'].fillna(2)
gdf_edges = gdf_edges.drop('ref', axis=1)

In [9]:
gdf_edges = gdf_edges.drop('name', axis=1)

In [10]:
gdf_edges = gdf_edges.fillna("unspecified")

/var/folders/bm/ftw8z7yx4x7__tj58fqqzvcm0000gn/T/ipykernel_9038/3639905595.py:1: DeprecationWarning: ExtensionArray.fillna added a 'copy' keyword in pandas 2.1.0. In a future version, ExtensionArray subclasses will need to implement this keyword or an exception will be raised. In the interim, the keyword is ignored by GeometryArray.
  gdf_edges = gdf_edges.fillna("unspecified")


In [11]:
gdf_edges.isna().sum()

osmid       0
oneway      0
lanes       0
highway     0
reversed    0
length      0
maxspeed    0
geometry    0
access      0
junction    0
bridge      0
tunnel      0
width       0
dtype: int64

In [12]:
def get_first_element(lst):
    if isinstance(lst, list):
        return lst[0]
    else:
        return lst

gdf_edges['lanes'] = gdf_edges['lanes'].apply(lambda x: get_first_element(x)).astype(int)
gdf_edges['reversed'] = gdf_edges['reversed'].apply(lambda x: get_first_element(x))
gdf_edges['maxspeed'] = pd.to_numeric(gdf_edges['maxspeed'].apply(lambda x: get_first_element(x)), errors='coerce').fillna(50).astype(int)
gdf_edges['reversed'] = gdf_edges['reversed'].map({True: 1, False: 0}).astype(int)
gdf_edges['oneway'] = gdf_edges['oneway'].map({True: 1, False: 0}).astype(int)

In [13]:
gdf_edges

osmid  \
u           v           key                                                    
26063726    31717945    0                                            4895484   
            29273144    0    [1014952979, 1014952978, 1007148203, 148749604]   
26063848    1391593855  0       [13982288, 125098266, 896320940, 1203516007]   
            258495263   0                                           23853023   
            26083906    0                                          715903941   
...                                                                      ...   
11795306899 11795306883 0                                         1270353666   
11795306921 11795306923 0                                         1270353669   
11795306923 11795306892 0                                         1270353667   
            11795306888 0                                         1270353667   
            11795306921 0                                         1270353669   

                             oneway  lanes        highway  reversed   length  \
u           v           key                                                    
26063726    31717945    0         1      4      secondary         0   25.021   
            29273144    0         1      4        primary         0  336.457   
26063848    1391593855  0         0      2    residential         0  996.078   
            258495263   0         0      2       tertiary         0  353.996   
            26083906    0         0      2    residential         0   66.590   
...                             ...    ...            ...       ...      ...   
11795306899 11795306883 0         0      2  living_street         1   89.591   
11795306921 11795306923 0         0      2  living_street         0   14.146   
11795306923 11795306892 0         0      2  living_street         0   23.305   
            11795306888 0         0      2  living_street         1   14.195   
            11795306921 0         0      2  living_street         1   14.146   

                             maxspeed  \
u           v           key             
26063726    31717945    0          50   
            29273144    0          60   
26063848    1391593855  0          50   
            258495263   0          50   
            26083906    0          50   
...                               ...   
11795306899 11795306883 0          20   
11795306921 11795306923 0          20   
11795306923 11795306892 0          20   
            11795306888 0          20   
            11795306921 0          20   

                                                                      geometry  \
u           v           key                                                      
26063726    31717945    0    LINESTRING (21.01745 52.15279, 21.01780 52.15271)   
            29273144    0    LINESTRING (21.01745 52.15279, 21.01745 52.152...   
26063848    1391593855  0    LINESTRING (20.99139 52.15757, 20.99129 52.157...   
            258495263   0    LINESTRING (20.99139 52.15757, 20.99127 52.157...   
            26083906    0    LINESTRING (20.99139 52.15757, 20.99147 52.157...   
...                                                                        ...   
11795306899 11795306883 0    LINESTRING (21.05158 52.28542, 21.05159 52.285...   
11795306921 11795306923 0    LINESTRING (21.05205 52.28449, 21.05205 52.284...   
11795306923 11795306892 0    LINESTRING (21.05212 52.28437, 21.05212 52.284...   
            11795306888 0    LINESTRING (21.05212 52.28437, 21.05211 52.284...   
            11795306921 0    LINESTRING (21.05212 52.28437, 21.05207 52.284...   

                                  access     junction       bridge  \
u           v           key                                          
26063726    31717945    0    unspecified  unspecified  unspecified   
            29273144    0    unspecified  unspecified  unspecified   
26063848    1391593855  0    unspecified  unspecified  unspecified   
            258495263   0    unspecifi

In [14]:
gdf_edges.index

MultiIndex([(   26063726,    31717945, 0),
            (   26063726,    29273144, 0),
            (   26063848,  1391593855, 0),
            (   26063848,   258495263, 0),
            (   26063848,    26083906, 0),
            (   26063848,  5274373939, 0),
            (   26063857,  1439574476, 0),
            (   26063857,  1391593855, 0),
            (   26063857,  3375221426, 0),
            (   26063923,  7093782406, 0),
            ...
            (11795306883, 11795306899, 0),
            (11795306888, 11795306923, 0),
            (11795306892, 11145761272, 0),
            (11795306892, 11795306883, 0),
            (11795306892, 11795306923, 0),
            (11795306899, 11795306883, 0),
            (11795306921, 11795306923, 0),
            (11795306923, 11795306892, 0),
            (11795306923, 11795306888, 0),
            (11795306923, 11795306921, 0)],
           names=['u', 'v', 'key'], length=44177)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
gdf_edges['highway'] = gdf_edges['highway'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
vectorizer = CountVectorizer(tokenizer=lambda x: x.split())


result_df = gdf_edges
for col in ['highway','access', 'junction', 'bridge', 'tunnel']:
    gdf_edges[col] = gdf_edges[col].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
    X2 = vectorizer.fit_transform(gdf_edges[col])
    df_count2 = pd.DataFrame(X2.toarray(), columns=vectorizer.get_feature_names_out())
    df_count2['index'] = gdf_edges.index
    df_count2.set_index("index", inplace = True)
    new_index_tuples = [(u, v, x) for u, v, x in gdf_edges.index]
    new_index = pd.MultiIndex.from_tuples(new_index_tuples, names=['u', 'v', 'key'])
    df_count2.index = new_index
    df_count2 = df_count2.rename(columns={col_nm: col + "_"+ col_nm for col_nm in df_count2.columns})
    result_df = pd.merge(result_df, df_count2, left_index=True, right_index=True)
result_df = result_df.drop(['highway','access', 'junction', 'bridge', 'tunnel'], axis=1)
result_df

/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/martynagrygiel/Documents/studia/gradient/

osmid  \
u           v           key                                                    
26063726    31717945    0                                            4895484   
            29273144    0    [1014952979, 1014952978, 1007148203, 148749604]   
26063848    1391593855  0       [13982288, 125098266, 896320940, 1203516007]   
            258495263   0                                           23853023   
            26083906    0                                          715903941   
...                                                                      ...   
11795306899 11795306883 0                                         1270353666   
11795306921 11795306923 0                                         1270353669   
11795306923 11795306892 0                                         1270353667   
            11795306888 0                                         1270353667   
            11795306921 0                                         1270353669   

                             oneway  lanes  reversed   length  maxspeed  \
u           v           key                                               
26063726    31717945    0         1      4         0   25.021        50   
            29273144    0         1      4         0  336.457        60   
26063848    1391593855  0         0      2         0  996.078        50   
            258495263   0         0      2         0  353.996        50   
            26083906    0         0      2         0   66.590        50   
...                             ...    ...       ...      ...       ...   
11795306899 11795306883 0         0      2         1   89.591        20   
11795306921 11795306923 0         0      2         0   14.146        20   
11795306923 11795306892 0         0      2         0   23.305        20   
            11795306888 0         0      2         1   14.195        20   
            11795306921 0         0      2         1   14.146        20   

                                                                      geometry  \
u           v           key                                                      
26063726    31717945    0    LINESTRING (21.01745 52.15279, 21.01780 52.15271)   
            29273144    0    LINESTRING (21.01745 52.15279, 21.01745 52.152...   
26063848    1391593855  0    LINESTRING (20.99139 52.15757, 20.99129 52.157...   
            258495263   0    LINESTRING (20.99139 52.15757, 20.99127 52.157...   
            26083906    0    LINESTRING (20.99139 52.15757, 20.99147 52.157...   
...                                                                        ...   
11795306899 11795306883 0    LINESTRING (21.05158 52.28542, 21.05159 52.285...   
11795306921 11795306923 0    LINESTRING (21.05205 52.28449, 21.05205 52.284...   
11795306923 11795306892 0    LINESTRING (21.05212 52.28437, 21.05212 52.284...   
            11795306888 0    LINESTRING (21.05212 52.28437, 21.05211 52.284...   
            11795306921 0    LINESTRING (21.05212 52.28437, 21.05207 52.284...   

                             width  highway_living_street  highway_primary  \
u           v           key                                                  
26063726    31717945    0      2.0                      0                0   
            29273144    0      2.0                      0                1   
26063848    1391593855  0      2.0                      0                0   
            258495263   0      2.0                      0                0   
            26083906    0      2.0                      0                0   
...                            ...                    ...              ...   
11795306899 11795306883 0      2.0                      1                0   
11795306921 11795306923 0      2.0                      1                0   
11795306923 11795306892 0      2.0                      1                0   
            11795306888 0      2.0                      1                0   
            11795306921 0      2.0                 

In [16]:
result_df.dtypes

osmid                          object
oneway                          int64
lanes                           int64
reversed                        int64
length                        float64
maxspeed                        int64
geometry                     geometry
width                         float64
highway_living_street           int64
highway_primary                 int64
highway_primary_link            int64
highway_residential             int64
highway_secondary               int64
highway_secondary_link          int64
highway_tertiary                int64
highway_tertiary_link           int64
highway_trunk                   int64
highway_trunk_link              int64
highway_unclassified            int64
access_delivery                 int64
access_designated               int64
access_destination              int64
access_no                       int64
access_permissive               int64
access_permit                   int64
access_unspecified              int64
access_yes  

## Node aggregation version

In [17]:
graph_embedder = OSMnxGraph(gdf_acc, gdf_nodes, gdf_edges)

graph_data = graph_embedder.create_graph(aggregation_type='node')
graph_data

/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply

Data(x=[19365, 8], edge_index=[2, 44177], y=[19365], street_count=[19365], highway=[19365], ref=[19365], accidents_count=[19365], osmid=[44177], oneway=[44177], lanes=[44177], edge_highway=[44177], reversed=[44177], length=[44177], maxspeed=[44177], geometry=[44177], access=[44177], junction=[44177], bridge=[44177], tunnel=[44177], width=[44177], crs=epsg:4326)

In [9]:
graph_embedder.show_statistics()

{'Nodes': 19365,
 'Edges': 44177,
 'Nodes dim': 8,
 'Nodes class': 2,
 'Directed': True,
 'Graph density [%]': 0.012}

In [10]:
graph_data.num_features

8

In [11]:
len(graph_data.y.unique())

2

In [17]:
from src.baseline_models.GCN.supervised_node_classification import SupervisedNodeClassificationGNN
from src.baseline_models.GCN.gcn import GCNModel
import torch

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_features = graph_data.num_features
hidden_dim = 256
out_dim = 128
num_classes = len(graph_data.y.unique())

gnn = GCNModel(in_dim=num_features, hidden_dim=hidden_dim, out_dim=out_dim)

model = SupervisedNodeClassificationGNN(gnn=gnn, emb_dim=out_dim, num_classes=num_classes, lr=0.0001)

In [19]:
graph_data.x

tensor([[5., 0., 0.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [3., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [3., 0., 0.,  ..., 0., 0., 0.]])

In [20]:
graph_data

Data(x=[19365, 8], edge_index=[2, 44177], y=[19365], street_count=[19365], highway=[19365], ref=[19365], accidents_count=[19365], osmid=[44177], oneway=[44177], lanes=[44177], name=[44177], edge_highway=[44177], reversed=[44177], length=[44177], maxspeed=[44177], geometry=[44177], access=[44177], junction=[44177], bridge=[44177], tunnel=[44177], edge_ref=[44177], width=[44177], crs=epsg:4326)

In [21]:
graph_data.node_attrs

<bound method BaseData.node_attrs of Data(x=[19365, 8], edge_index=[2, 44177], y=[19365], street_count=[19365], highway=[19365], ref=[19365], accidents_count=[19365], osmid=[44177], oneway=[44177], lanes=[44177], name=[44177], edge_highway=[44177], reversed=[44177], length=[44177], maxspeed=[44177], geometry=[44177], access=[44177], junction=[44177], bridge=[44177], tunnel=[44177], edge_ref=[44177], width=[44177], crs=epsg:4326)>

In [22]:
graph_data.edge_attr

In [23]:
features = graph_embedder.get_node_attrs()
features

/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,street_count,crossing,mini_roundabout,motorway_junction,speed_camera,traffic_signals,turning_circle,turning_loop
osmid,,,,,,,,
26063726,5,0,0,0,0,0,0,0
26063848,4,0,0,0,0,0,0,0
26063857,3,0,0,0,0,0,0,0
26063923,3,0,0,0,0,0,0,0
26083887,3,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
11795306888,1,0,0,0,0,0,0,0
11795306892,3,0,0,0,0,0,0,0
11795306899,1,0,0,0,0,0,0,0


In [24]:
type(features.index)

pandas.core.indexes.base.Index

In [25]:
len(set(features.index.to_list()))

19365

In [26]:
gdf_nodes.index

Index([   26063726,    26063848,    26063857,    26063923,    26083887,
          26083906,    26083915,    26083926,    26083927,    26083933,
       ...
       11784580284, 11784580288, 11784580291, 11787954936, 11795306883,
       11795306888, 11795306892, 11795306899, 11795306921, 11795306923],
      dtype='int64', name='osmid', length=19365)

In [27]:
len(graph_data.y)

19365

In [28]:
features.index

Index([   26063726,    26063848,    26063857,    26063923,    26083887,
          26083906,    26083915,    26083926,    26083927,    26083933,
       ...
       11784580284, 11784580288, 11784580291, 11787954936, 11795306883,
       11795306888, 11795306892, 11795306899, 11795306921, 11795306923],
      dtype='int64', name='osmid', length=19365)

In [29]:
features.index.unique()

Index([   26063726,    26063848,    26063857,    26063923,    26083887,
          26083906,    26083915,    26083926,    26083927,    26083933,
       ...
       11784580284, 11784580288, 11784580291, 11787954936, 11795306883,
       11795306888, 11795306892, 11795306899, 11795306921, 11795306923],
      dtype='int64', name='osmid', length=19365)

In [30]:
from sklearn.model_selection import train_test_split
nodes_labels = {'node': features.index.to_numpy(), 'label': graph_data.y.cpu().numpy()}
df_to_split = pd.DataFrame(nodes_labels)

df_train, df_test = train_test_split(df_to_split, test_size=0.2, random_state=42, stratify=df_to_split['label'])
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=42, stratify=df_test['label'])

train_nodes = df_train['node']
val_nodes = df_val['node']
test_nodes = df_test['node']

train_mask = []   
val_mask = []
test_mask = []

for i in range(len(graph_data.y)):

  if i in train_nodes:
    train_mask.append(True)
    val_mask.append(False)
    test_mask.append(False)
  elif i in val_nodes:
    train_mask.append(False)
    val_mask.append(True)
    test_mask.append(False)
  elif i in test_nodes:
    train_mask.append(False)
    val_mask.append(False)
    test_mask.append(True)

graph_data.train_mask = torch.tensor(train_mask).cpu()
graph_data.val_mask = torch.tensor(val_mask).cpu()
graph_data.test_mask = torch.tensor(test_mask).cpu()

In [ ]:
import pytorch_lightning as pl
from src.baseline_models.GCN.GraphData import GraphData
trainer = pl.Trainer(max_epochs=50)
datamodule = GraphData([graph_data])
trainer.fit(model=model, datamodule=datamodule)

In [32]:
trainer.test(model=model, datamodule=datamodule, verbose=False)

/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  9.74it/s]


/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[{'step': 50.0,
  'test/auc_weighted': 0.4094811677932739,
  'test/f1_weighted': 0.7625361680984497,
  'test/precision_weighted': 0.7003365159034729,
  'test/recall_weighted': 0.8368611335754395,
  'test/accuracy_weighted': 0.8368611335754395}]

In [33]:
test_auc = trainer.test(model=model, datamodule=datamodule, verbose=False)[0]["test/auc_weighted"]

z, y, y_pred = trainer.predict(model=model, datamodule=datamodule)[0]

print(f'AUC test = {test_auc * 100.:.2f}[%]')

/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 13.01it/s]

/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 21.69it/s]
AUC test = 40.95[%]


In [34]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [35]:
y_pred_test = y_pred[test_mask]

In [36]:
y_pred_test.max()

0

In [37]:
print(y[test_mask].numpy())

[0 0 0 ... 0 1 0]


In [38]:
len(y)

19365

## Edge aggregation version

In [35]:
graph_embedder = OSMnxGraph(gdf_acc, gdf_nodes, gdf_edges)

graph_data = graph_embedder.create_graph(aggregation_type='edge')
graph_data

osmid                object
oneway                int64
lanes                 int64
highway              object
reversed              int64
length              float64
maxspeed              int64
geometry           geometry
access               object
junction             object
bridge               object
tunnel               object
width               float64
accidents_count       int64
dtype: object


/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


<class 'torch_geometric.data.data.Data'>


/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Data(x=[44177, 36], edge_index=[2, 44177], y=[44177], street_count=[19365], highway=[19365], ref=[19365], osmid=[44177], oneway=[44177], lanes=[44177], name=[44177], edge_highway=[44177], reversed=[44177], length=[44177], maxspeed=[44177], geometry=[44177], access=[44177], junction=[44177], bridge=[44177], tunnel=[44177], edge_ref=[44177], width=[44177], accidents_count=[44177], crs=epsg:4326)

In [25]:
graph_embedder.show_statistics()

{'Nodes': 44177,
 'Edges': 44177,
 'Nodes dim': 36,
 'Nodes class': 2,
 'Directed': True,
 'Graph density [%]': 0.002}

In [26]:
features = graph_embedder.get_edge_attrs()

                                                                       osmid  \
u           v           key                                                    
26063726    31717945    0                                            4895484   
            29273144    0    [1014952979, 1014952978, 1007148203, 148749604]   
26063848    1391593855  0       [13982288, 125098266, 896320940, 1203516007]   
            258495263   0                                           23853023   
            26083906    0                                          715903941   
...                                                                      ...   
11795306899 11795306883 0                                         1270353666   
11795306921 11795306923 0                                         1270353669   
11795306923 11795306892 0                                         1270353667   
            11795306888 0                                         1270353667   
            11795306921 0               

/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/martynagrygiel/Documents/studia/gradient/

In [21]:
from src.baseline_models.GCN.supervised_node_classification import SupervisedNodeClassificationGNN
from src.baseline_models.GCN.gcn import GCNModel
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_features = graph_data.num_features
hidden_dim = 256
out_dim = 128
num_classes = len(graph_data.y.unique())

gnn = GCNModel(in_dim=num_features, hidden_dim=hidden_dim, out_dim=out_dim)

model = SupervisedNodeClassificationGNN(gnn=gnn, emb_dim=out_dim, num_classes=num_classes, lr=0.0001)

from sklearn.model_selection import train_test_split
nodes_labels = {'node': features.index.to_numpy(), 'label': graph_data.y.cpu().numpy()}
df_to_split = pd.DataFrame(nodes_labels)

df_train, df_test = train_test_split(df_to_split, test_size=0.2, random_state=42, stratify=df_to_split['label'])
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=42, stratify=df_test['label'])

train_nodes = df_train['node']
val_nodes = df_val['node']
test_nodes = df_test['node']

train_mask = []   
val_mask = []
test_mask = []

for i in range(len(graph_data.y)):

  if i in train_nodes:
    train_mask.append(True)
    val_mask.append(False)
    test_mask.append(False)
  elif i in val_nodes:
    train_mask.append(False)
    val_mask.append(True)
    test_mask.append(False)
  elif i in test_nodes:
    train_mask.append(False)
    val_mask.append(False)
    test_mask.append(True)

graph_data.train_mask = torch.tensor(train_mask).cpu()
graph_data.val_mask = torch.tensor(val_mask).cpu()
graph_data.test_mask = torch.tensor(test_mask).cpu()


import pytorch_lightning as pl
from src.baseline_models.GCN.GraphData import GraphData
trainer = pl.Trainer(max_epochs=50)
datamodule = GraphData([graph_data])
trainer.fit(model=model, datamodule=datamodule)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

  | Name                 | Type       | Params
----------------------------------------------------
0 | _gnn                 | GCNModel   | 42.1 K
1 | _classification_head | Sequential | 16.8 K
2 | _loss_fn             | NLLLoss    | 0     
---------------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 44177. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 2:   0%|          | 0/1 [00:00<?, ?it/s, v_num=1]        

...

loss: 0.31178414821624756
Epoch 49:   0%|          | 0/1 [00:00<?, ?it/s, v_num=1]        

/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


loss: 0.31912755966186523
Epoch 49: 100%|██████████| 1/1 [00:01<00:00,  0.81it/s, v_num=1]

/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 1/1 [00:01<00:00,  0.79it/s, v_num=1]


In [22]:
trainer.test(model=model, datamodule=datamodule, verbose=False)

/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  8.22it/s]


/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[{'step': 50.0,
  'test/auc_weighted': 0.5558940172195435,
  'test/f1_weighted': 0.8851123452186584,
  'test/precision_weighted': 0.8507536053657532,
  'test/recall_weighted': 0.9223629832267761,
  'test/accuracy_weighted': 0.9223629832267761}]

In [23]:
test_auc = trainer.test(model=model, datamodule=datamodule, verbose=False)[0]["test/auc_weighted"]

z, y, y_pred = trainer.predict(model=model, datamodule=datamodule)[0]

print(f'AUC test = {test_auc * 100.:.2f}[%]')

/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  8.41it/s]

/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/martynagrygiel/Documents/studia/gradient/gradient/env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 16.10it/s]
AUC test = 55.59[%]


In [24]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [25]:
y_pred.max()

0